# ShExStatements

ShExStatements is a simpler way to write ShEx statements. It makes use of triples and quadruples, separated by | to write shape expressions. Though not as expressive as the complete ShEx, its goal is to reduce the learning curve of new comers of ShEx language. 

## Examples
Take for example the following ShEx statements describe the shape expression of a painting on Wikidata.
```painting|P31|Q3305213
   painting|P571|xsd:dateTime
   painting|P276|.|+
   painting|P1476|.|+ 
   painting|P195|.|+
   painting|P170|creator|+
   creator|P31|.|+```
 
 On calling the function `generate_shex` with the above string will generate the following shape expression.
 
```
   PREFIX wd: <http://www.wikidata.org/entity/>
   PREFIX wdt: <http://www.wikidata.org/prop/direct/>
   start = @painting
   painting {
     wdt:P31 wd:Q3305213 ;
     wdt:P571 @xsd:dateTime  ;
     wdt:P276  .  + ;
     wdt:P1476  .  ;
     wdt:P195  .  + ;
     wdt:P170 @creator  + ;
   }
   creator {
    wdt:P31  .  + ;
   }
   ```
 
ShExStatements is inspired by QuickStatements used for creating statements on Wikidata.

In [3]:
def generate_shex(shexstatement):
    start = None
    nodeconstraints = {}
    for line in shexstatement.split("\n"):
        terms = line.split("|")
        if len(terms) < 3:
            continue
        else:
            if terms[0] not in nodeconstraints:
                nodeconstraints[terms[0]] = []
            if start is None:
                start = terms[0]
            combination = []
            term2 = None
            if terms[1].startswith("P"):
                term2 = "wdt:" + terms[1]
                combination.append(term2)
            term3 = terms[2]
            if term3.startswith("Q"):
                term3 = "wd:" + term3
            elif(term3 == "."):
                term3 = " " + term3 + " "
            else:
                term3 = "@" + term3 + " "
            if( "," in term3):
                term3valuelist = []
                for term3value in term3.split(","):
                    if term3value.startswith("Q"):
                        term3valuelist.append("wd:" + term3value)
                    else:
                        term3valuelist.append(term3value)
                term3 = "[ " + " ".join(term3valuelist) + " ]"
            combination.append(term3)
            cardinality = None
            allowedvalues = {"+", "*"}
            if (len(terms) > 3):
                if terms[3] in allowedvalues:
                    cardinality = terms[3]
                    combination.append(cardinality)
            nodeconstraints[terms[0]].append(combination)
    print("PREFIX wd: <http://www.wikidata.org/entity/>")
    print("PREFIX wdt: <http://www.wikidata.org/prop/direct/>")
    if start is not None:
        print("start = @" + start)

    for key in nodeconstraints.keys():
        print(key + " {" )
        for combination in nodeconstraints[key]:
            print("  " +" ".join(combination) + " ;")
        print("}")

In [4]:
shexstatement = """
painting|P31|Q3305213
painting|P571|xsd:dateTime
painting|P276|.|+
painting|P1476|.|+ 
painting|P195|.|+
painting|P170|creator|+
creator|P31|.|+
"""
generate_shex(shexstatement)

PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
start = @painting
painting {
  wdt:P31 wd:Q3305213 ;
  wdt:P571 @xsd:dateTime  ;
  wdt:P276  .  + ;
  wdt:P1476  .  ;
  wdt:P195  .  + ;
  wdt:P170 @creator  + ;
}
creator {
  wdt:P31  .  + ;
}
